## 业界分词面临的难点

- 分词的规范
- 歧义词
- 未登录词

## 基于词典的分词方法
&emsp;&emsp;首先我们需要有一个收集或者整理的词典，然后将文章分成句子，再对句子进行分词。

&emsp;&emsp;这种基于词典的分词方法，天生就不能解决未登录词和歧义词问题（歧义词问题是碰巧解决了大部分，毕竟歧义词在文章中的占比并不大）。
###  一、最长匹配算法
#### 1、正向最长词匹配
&emsp;&emsp;从左侧就是正向，即从左侧进行最长词的匹配。

&emsp;&emsp;例如：存在词典['小米','手机品牌']，需要将"小米是手机品牌"进行分词，我们第一步可以直接将句子进行词匹配，如果匹配不成功则从右边减掉一个字，再匹配，依次类推直到匹配到一个词或者只剩一个字；然后将该词去掉，再不断重复该步骤，直到该句子处理完，匹配出所有词。

&emsp;&emsp;我们看下演示就明白了


> 小米是手机品牌 $\Longrightarrow$ 匹配失败 \
> 小米是手机品 $\Longrightarrow$ 匹配失败 \
> 小米是手机 $\Longrightarrow$ 匹配失败 \
> 小米是手 $\Longrightarrow$ 匹配失败 \
> 小米是 $\Longrightarrow$ 匹配失败 \
> 小米 $\Longrightarrow$ 匹配成功 \
>  \
> 是手机品牌 $\Longrightarrow$ 匹配失败 \
> 是手机品 $\Longrightarrow$ 匹配失败 \
> 是手机 $\Longrightarrow$ 匹配失败 \
> 是手 $\Longrightarrow$ 匹配失败 \
> 是 $\Longrightarrow$ 只剩最后一个字 \
>  \
> 手机品牌 $\Longrightarrow$ 匹配成功

看下代码：

In [9]:
dictionary = ['小米', '手机品牌']
sentence = '小米是手机品牌'


def seg_left(dictionary, sentence):
    result = []
    while len(sentence) > 1:
        for end in range(len(sentence), 0, -1):
            if sentence[:end] in dictionary or end == 1:
                word = sentence[:end]
                result.append(word)
                sentence = sentence[end:]
                # print(word, sentence)
                break
    
    return result            

       
print(seg_left(dictionary, sentence))

['小米', '是', '手机品牌']


你会发现如果词典是['研究','生命','研究生','起源']，句子是"研究生命起源"，那正向匹配就会出现问题，结果为：'研究生'、'命'、'起源'。所以能够看出正向最长匹配也是有问题的。解决不掉这种歧义词的问题

#### 2、逆向最长词匹配
&emsp;&emsp;同理，逆向最长词匹配和正向最长词匹配道理是一样的，它是从右侧开始进行最长词匹配，当然也同样存在问题，解决不掉这种歧义词的问题
例如词典是['标签','项目','目的','研究']，句子是"标签项目的研究"，结果为：'标签'、'项'、'目的'、'研究'

#### 3、双向最长词匹配
&emsp;&emsp;双向最长词匹配充分利用了前两者的优势。它同时执行正向和逆向最长词匹配，若两者的词个数不同，则返回词个数更少的那一个。
否则，返回两者中单字更少的那一个。当单字数也相同时，优先返回逆向最长匹配的结果。
&emsp;&emsp;即使是双向最长词匹配也存在问题，解决不掉所有歧义词的问题。当然它解决了正向或反向的部分问题，例如：句子"研究生命起源"，可以分出'研究'、'生命'、'起源'这样正确的词，但是对于"标签项目的研究"这样的句子也存在错误的分词情况，分词为'标签'、'项'、'目的'、'研究'

#### 总结
&emsp;&emsp;所以能够看出这种匹配分词的方式还是存在问题的，并不能解决所有歧义的问题，当然双向最长词匹配是三者中较好的。
&emsp;&emsp;据SunM.S. 和 Benjamin K.T.（1995）的研究表明，中文中90.0％左右的句子，正向最大匹配法和逆向最大匹配法完全重合且正确，只有大概9.0％的句子两种切分方法得到的结果不一样，但其中必有一个是正确的，只有不到1.0％的句子，或者正向最大匹配法和逆向最大匹配法的切分虽重合却是错的，或者正向最大匹配法和逆向最大匹配法切分不同但两个都不对。这正是双向最大匹配法在实用中文信息处理系统中得以广泛使用的原因所在。

#### 分词性能提升--字典树
&emsp;&emsp;当数据量大时，需要对这种分词方式进行性能提升，最大的瓶颈就在于判断一个词是否在词典中。如果用有序集合的话，复杂度是o(logn) ( n是词典大小);如果用散列表(字典)的话，时间复杂度虽然下降了，但内存复杂度却上去了。有没有速度又快、内存又省的数据结构呢？这就是字典树。

&emsp;&emsp;字符串集合常用字典树(trie树、前缀树)存储，这是一种字符串上的树形数据结构。字典树中每条边都对应一个字， 从根节点往下的路径构成一个个字符串。字典树并不直接在节点上存储字符串， 而是将词语视作根节点到某节点之间的一条路径，并在终点节点(蓝色) 上做个标记“该节点对应词语的结尾”。字符串就是一 条路径，要查询一个单词，只需顺着这条路径从根节点往下走。如果能走到特殊标记的节点，则说明该字符串在集合中，否则说明不存在。

&emsp;&emsp;下面是字典树的代码演示，用以理解字典树这种数据存储结构

In [ ]:
import collections
 
class Node(object):
    def __init__(self):
        # 通过Key访问字典，当Key不存在时，会引发‘KeyError’异常。
        # 为了避免这种情况的发生，可以使用collections类中的defaultdict()方法来为字典提供默认值
        self.children = collections.defaultdict(Node)
        # root2node is a word
        self.isword = False
        # root2node present the word
        self.word = None
        # root2node node count
        self.count = 0
 
class Trie(object):
    def __init__(self):
        self.root = Node()
    
    def addWord(self, word):
        # get root of trie
        current = self.root
        for w in word:
            # create a child, count + 1
            current = current.children[w]
            current.count += 1
        current.isword = True
        current.word = word
    
    def addWords(self, words):
        for word in words:
            self.addWord(word)
    
    def hasWord(self, word):
        current = self.root
        for w in word:
            # choose the w-node in children
            current = current.children.get(w)
            if current == None:
                return False
        return current.isword
 
    def startWith(self, prefix):
        current = self.root
        for p in prefix:
            current = current.children.get(p)
            if current == None:
                return False
        return True
    
    def delWord(self, word):
        mission = False
        if self.hasWord(word):
            mission = True
            current = self.root
            for w in word:
                wNode = current.children.get(w)
                # delete a node count, if the number of current node is 0, delete it
                wNode.count -= 1
                if wNode.count == 0:
                    # current is Node object, but current.children is dict object
                    # del current will not change [global variable t], though they own the same memory address
                    current.children.pop(w)
                    break
                current = wNode
        return mission
    
    def delTrie(self):
        nodes = self.root.children
        for k in list(nodes.keys()):
            if k is not None:
                nodes.pop(k)
        del self.root
    
    def printTrie(self, n):
        for c in n.children:
            print(c)
            current = n.children.get(c)
            if current is None:
                return
            if current.isword:
                print(current.word)
            self.printTrie(current)
trie = Trie()
trie.addWords(['word', 'or', 'work'])
trie.printTrie(trie.root)

w
o
r
d
word
k
work
o
r
or


<img src="./imgs/trie.png" alt="字典树" width="150" height="200" align="left"/>

绿色表示以上路径是一个单词；

通过代码和图我们应该能理解字典树的原理了。

### 二、最短路径算法
#### 1、N最短路径分词
&emsp;&emsp;最短路径算法是使用图论中的迪克斯特拉算法(Dijkstra算法)进行实现。解决的问题是从一个顶点到其余各顶点的最短路径算法，解决的是有权图中最短路径问题。一会再聊Dijkstra算法。

&emsp;&emsp;应用到分词上，就是将一个句子切分成若干单字，并将每个字连接起来，对于存在于字典中的词（切词使用匹配所有可能词的方式）可以进行连接，然后求从第一个字到最后一个字所有路径中的最短路径。而词之间的距离却可以衡量词与词同时出现的概率，这样就能解决掉大部分的歧义问题。

假设：词典为['的确','确实','实在','在理']，句子为"他说的确实在理"

来看图理解下：

<img src="./imgs/dijkstra.png" alt="最短路径" width="500" height="200" align="left"/>

线上的1代表这个词或者字的距离，所以我们能够得出从”他“到”理“的路径有哪些，且距离都是多少

0 $\longrightarrow$ 1 $\longrightarrow$ 2 $\longrightarrow$ 4 $\longrightarrow$ 6 $\longrightarrow$ 7 距离为5

0 $\longrightarrow$ 1 $\longrightarrow$ 2 $\longrightarrow$ 3 $\longrightarrow$ 5 $\longrightarrow$ 7 距离为5

0 $\longrightarrow$ 1 $\longrightarrow$ 2 $\longrightarrow$ 4 $\longrightarrow$ 5 $\longrightarrow$ 7 距离为5

可以看出它们的距离都是一样的，如果有三个字的词距离就不会都一样了。但是很明显这种距离都一样不是我很想要的，因为我们要计算出到底哪一种最合理。下面我们利用马尔科夫模型计算词之间的距离。

> **马尔科夫链** \
> &emsp;&emsp;马尔科夫链是指具有马尔可夫性质的随机过程；在过程中，在给定当前信息的情况下，过去的信息状态对于预测将来状态是无关的。\

>我们知道句子里的词之间都是有关系，换句话说就是一个词的出现是和前面的很多词有关系，因为如果我们计算一个词出现的概率由前面多个词影响那计算量太大了，为了简化，我们应用马尔科夫链，它的理论就是一个事件的出现只和前一个事件有关，在这里也就是一个词的出现只由它的前一个词影响，这样就大大减少了计算量，同时也解决了数据稀疏的问题，当一个句子过长时，可能统计不到词频。

计算过程：

&emsp;&emsp;P(的确|说) = $\frac{C(说的确)}{C(说)}$

&emsp;&emsp;解释下，P(的确|说)就是在“说”出现的情况下，“的确”出现的概率，C(说的确)就是“说的确”这个字符串出现的词频（所有的你统计的文本里的）；C(说)就是“说”的词频。

&emsp;&emsp;因为概率值越大代表可能性越大，所以取个倒数作为距离即可，又因为概率可能非常小从而导致距离非常大，所以取个log。

&emsp;&emsp;最终距离公式为：

&emsp;&emsp;distance = $\log(\frac{1}{P(的确|说)})$